In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.stats import randint
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

In [5]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_ids = test['id']

Label_encoder = LabelEncoder()

data['Sex_encoded'] = Label_encoder.fit_transform(data['Sex'])
data.drop('Sex', axis=1, inplace=True)

test['Sex_encoded'] = Label_encoder.fit_transform(test['Sex'])
test.drop('Sex', axis=1, inplace=True)

y = data['Rings']
X = data.drop('Rings', axis = 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Scale the features for KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)  # Scale validation set as well

# Create KNeighborsRegressor
knn_regressor = KNeighborsRegressor()

# Define parameter grid for GridSearchCV
param_grid_knn = {
    'n_neighbors': [3, 5, 10, 20],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weighting scheme for neighbors
    'p': [1, 2]  # Power parameter for the Minkowski distance (1 for Manhattan, 2 for Euclidean)
}

# Create GridSearchCV object
grid_search_knn = GridSearchCV(estimator=knn_regressor, param_grid=param_grid_knn, 
                                scoring='neg_mean_squared_error', cv=5)

# Perform grid search
grid_search_knn.fit(X_train_scaled, y_train)

# Get best estimator
best_knn_regressor = grid_search_knn.best_estimator_

# Print best parameters and score
print("Best Parameters for KNN Regressor:", grid_search_knn.best_params_)
print("Best Score (Negative Mean Squared Error) for KNN Regressor:", grid_search_knn.best_score_)


Best Parameters for KNN Regressor: {'n_neighbors': 20, 'p': 2, 'weights': 'distance'}
Best Score (Negative Mean Squared Error) for KNN Regressor: -3.8132395980800817


In [9]:
predictions = best_knn_regressor.predict(X_val)

# Ensure predictions are non-negative
predictions = np.clip(predictions, 0, None)

# Calculate RMSLE
rmsle = np.sqrt(mean_squared_log_error(np.log1p(predictions), np.log1p(y_val)))
print("RMSLE Score:", rmsle)

c:\Users\Beheerder\anaconda3\envs\block_c_new\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


RMSLE Score: 0.09439367283560107


In [10]:
submission_preds = best_knn_regressor.predict(test)

# Ensure submission predictions are non-negative
submission_preds = np.clip(submission_preds, 0, None)

# Round the predictions and convert them to integers
submission_preds_rounded = np.round(submission_preds).astype(int)

c:\Users\Beheerder\anaconda3\envs\block_c_new\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


In [11]:
df = pd.DataFrame({'id': test_ids.values, 
                   'Rings': submission_preds_rounded
                   })

In [14]:
df.to_csv('submission4.csv', index=False)

In [13]:
# Check if any target value is negative
negative_targets_exist = (df['Rings'] < 0).any()

if negative_targets_exist:
    print("Negative target values exist.")
else:
    print("No negative target values.")


No negative target values.
